<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*

## Imports

In [7]:
import praw
from openai import OpenAI

In [2]:
reddit = praw.Reddit(client_id="TODO", # the App ID retreived from the reddit application
                     client_secret="TODO", # the secret retreived from the reddit application
                     user_agent="sentiment analysis test (by u/YourUserName)"  # add your reddit name
                    )

Version 7.6.1 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


We can now already use our reddit instance to obtain public information from reddit!

In [3]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)


test
test
Test Submission
test for photos
test nerd
Test
T234
A test post
test
test


Of course we want to access hot subreddits like r/wallstreetbets or r/stocks

In [4]:
subreddit_stocks = reddit.subreddit("stocks")

In [5]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
3192


Let's say we want to access the first 5 hot posts and three most upvoted comments:

In [6]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    c = 0
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]":
            print(comment)
            c+=1
        if c == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread September 2023
My [last post](https://www.reddit.com/r/stocks/comments/11ezwar/rate_my_portfolio_rstocks_quarterly_thread_march/jah0b73/) was 6 months ago.

To recap, a brief overview of my strategy, that has not since changed:

> I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much.

> I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an

Ok fine, but all cases heard by a jury will result in real consequences rather than just a fine or disgorgement. Right now the SEC determines if you broke their rules and slaps you on the wrist with a fine. If these types of things go in front of a jury i want to see a conviction of fraud and jail time.
Yeah bad idea. It will be too easy for technical and complex cases to confound layman jurors.
US economic growth for last quarter is revised up to a 5.2% annual rate
Bears in shambles
Economy can handle 6%

Do it jay
Why do you think Warren Buffett hates Nike stock ($NKE) so much?
Probably because the shoe and clothing industry is one of the hardest and any brand tomorrow can be “ out of style “  . 

He doesn’t invest in what he doesn’t understand and he 100% isn’t into fashion
Hearing Munger speak in the Acquired podcast interview, he specifically said he (and therefore I assume Buffett) don’t like “style companies” in general, and Nike was specifically brought up as an example. In the

We can now use GPT to extract stock tickers from all of these posts and tell us whether the sentiment for the mentioned assets is positive, neutral or negative:

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.


In [8]:
client = OpenAI()

## Prompt Generation
First, we need to create a dictionary containing the title and top three comments

In [9]:
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [10]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [11]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [12]:
print(titles_and_comments[2])

Title: Why do you think Warren Buffett hates Nike stock ($NKE) so much?

Comments: 

Probably because the shoe and clothing industry is one of the hardest and any brand tomorrow can be “ out of style “  . 

He doesn’t invest in what he doesn’t understand and he 100% isn’t into fashion
Hearing Munger speak in the Acquired podcast interview, he specifically said he (and therefore I assume Buffett) don’t like “style companies” in general, and Nike was specifically brought up as an example. In the rare cases where they own apparel businesses, they seem more utilitarian in nature, less tied to brand appeal.

Nike has been an enduring brand for many years…but preferences can and do evolve in unpredictable ways. If you’re someone with no personal knowledge of the winds of fashion and consumer preference that affect a given brand (raising my hand here), it probably makes sense to avoid those brands in general.



In [16]:
def create_system_prompt():
    return "You are a helpful assistant for sentiment analysis."

def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments


In [17]:
system_prompt = create_system_prompt()
prompt = create_prompt(titles_and_comments[1])

In [18]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: US economic growth for last quarter is revised up to a 5.2% annual rate

Comments: 

Bears in shambles
Economy can handle 6%

Do it jay



In [22]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                                                messages=[
                                                    {"role": "system", "content": system_prompt},
                                                    {"role": "user", "content": prompt},
                                                ],
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response.choices[0].message.content)
    print("-"*30)

Title: US economic growth for last quarter is revised up to a 5.2% annual rate

Comments: 

Bears in shambles
Economy can handle 6%

Do it jay

Sentiment: Title: No company mentioned
Sentiment: Neutral

Comments:
1. No company mentioned
   Sentiment: Negative

2. No company mentioned
   Sentiment: Neutral

3. No company mentioned
   Sentiment: Neutral
------------------------------
Title: Why do you think Warren Buffett hates Nike stock ($NKE) so much?

Comments: 

Probably because the shoe and clothing industry is one of the hardest and any brand tomorrow can be “ out of style “  . 

He doesn’t invest in what he doesn’t understand and he 100% isn’t into fashion
Hearing Munger speak in the Acquired podcast interview, he specifically said he (and therefore I assume Buffett) don’t like “style companies” in general, and Nike was specifically brought up as an example. In the rare cases where they own apparel businesses, they seem more utilitarian in nature, less tied to brand appeal.

Nike